<a href="https://colab.research.google.com/github/mlukan/GDA3B2021/blob/main/NN/Stock_Prediction_Quest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introduction 
In this exercise you will work with stock market data. We use the data available at [Yahoo Finance](https://finance.yahoo.com/quote/DAX/history?period1=1478390400&period2=1604534400&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true)
You will build sequential models to predict the variations for Global X DAX Germany ETF (DAX). The goal is to predict whether the stock value is higher or lower than its actual value of previous day.

In [83]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

### Loading and Preprocessing Data

In [5]:
# Loading training data
#training_data = pd.read_csv("https://github.com/mlukan/GDA3B2021/tree/main/daxdata/dax_train_2019.csv.txt", sep=",")
training_data = pd.read_csv("../daxdata/dax_train_2019.csv.txt", sep=",")

print(f"Training data column info:\n {training_data.dtypes}")
print(f"Training data shape is {training_data.shape}")

Training data column info:
 date          object
open         float64
high         float64
low          float64
close        float64
adj_close    float64
volume         int64
dtype: object
Training data shape is (644, 7)


In [6]:
# Loading testing data
#testing_data = pd.read_csv("https://github.com/mlukan/GDA3B2021/tree/main/daxdata/dax_test_2019.csv.txt", sep=",")
testing_data = pd.read_csv("../daxdata/dax_test_2019.csv.txt", sep=",")

print(f"Testing data column info:\n {testing_data.dtypes}")
print(f"Testing data shape is {testing_data.shape}")

Testing data column info:
 date          object
open         float64
high         float64
Low          float64
close        float64
adj_close    float64
volume         int64
dtype: object
Testing data shape is (148, 7)


In [ ]:
len(testing_data)
testing_data.shape[0]

148

####  Are there any NaN values in training and testing data sets?

In [7]:

training_count_nan = training_data.shape[0] - training_data.count()
testing_count_nan = testing_data.shape[0] - testing_data.count()
print(f"Number of NaNs in training data columns:\n{training_count_nan}")
print(f"Number of NaNs in testing data columns:\n{testing_count_nan}")

Number of NaNs in training data columns:
date         0
open         0
high         0
low          0
close        0
adj_close    0
volume       0
dtype: int64
Number of NaNs in testing data columns:
date         0
open         0
high         0
Low          0
close        0
adj_close    0
volume       0
dtype: int64


In [92]:
testing_data.shape

(148, 7)

#### Preprocessing the data, extracting the features, and normalising the features using `preprocessing.MinMaxScaler` function from sklearn

We have provided you with the relevant function here. But we expect you to explore the data generated. 

What are the shapes?

Do a test whether features and labels correspond, i.e. whether the label for each sequence is the value following that sequence? 

Optional: Extend the function to include more features - and change it 
to choose a different column as target, i.e. adj_close


In [ ]:
# Limiting the training and testing data to "open" column 
training_data_processed = training_data.iloc[:, 1:2].values
testing_data_processed = testing_data.iloc[:, 1:2].values

In [9]:
def create_features_labels(training_data, testing_data, scaler=MinMaxScaler(feature_range = (0, 1)), sequence_length=60):
    """
    Create feature and labels from training data
    :param training_data
    :param testing_data
    :param sequence_length
    :param scaler
    :return: Features and labels sets, and labels bins
    """
    features_set = []
    labels = []
    labels_bin = []
    test_features = []
    # Limiting the training and testing data to relevant column (here "open" column)
    training_data_processed = training_data.iloc[:, 1:2].values
    # Normalization: Transform features by scaling each feature in training to be in a range of (0, 1)
    training_data_processed = scaler.fit_transform(training_data_processed)
    # Preparing training data features and labels
    for i in range(sequence_length, len(training_data)):
        features_set.append(training_data_processed[i-sequence_length:i, 0])
        labels.append(training_data_processed[i, 0])
        # Fill binary labels
        if training_data_processed[i,0] > training_data_processed[i-1,0]:
            labels_bin.append(1)
        else:
            labels_bin.append(0)
    # Preparing the testing data features
    total = pd.concat((training_data['open'], testing_data['open']), axis=0)
    test_inputs = total[len(total) - len(testing_data) - sequence_length:].values
    test_inputs = test_inputs.reshape(-1,1)
    test_inputs = scaler.transform(test_inputs)

    for i in range(sequence_length, len(test_inputs)):
        test_features.append(test_inputs[i-sequence_length:i, 0])
    
    test_features = np.array(test_features)
    test_features = np.reshape(test_features, (test_features.shape[0], test_features.shape[1], 1))
    features_set, labels, labels_bin = np.array(features_set), np.array(labels), np.array(labels_bin)
    return np.reshape(features_set, (features_set.shape[0], features_set.shape[1], 1)), labels, labels_bin, test_features, scaler

In [10]:
features_set, labels, labels_bin, test_features, scaler = create_features_labels(training_data, testing_data, MinMaxScaler(feature_range = (0, 1)), 60)

In [16]:
# What are the shapes?
for f in ['features_set', 'labels', 'labels_bin', 'test_features']:
    print(f,eval(f+".shape"))

features_set (584, 60, 1)
labels (584,)
labels_bin (584,)
test_features (148, 60, 1)


In [81]:


# Do a test whether features and labels correspond, i.e. whether the label for each sequence is the value following that sequence? 
ct=0
for f, l in  zip(features_set[1:,-1].flatten(),labels):
    if ct <10:
        print(f,l)
        ct+=1
    if f!=l:
        print(f,l,"does not match")
    

In [103]:
labels_bin=labels_bin.reshape(labels_bin.shape[0],1)

In [112]:
type(labels)

numpy.ndarray

### Model
In this section we ask you to build and train a LSTM neural network and use it predict the stock price variations on your testing data.

In [104]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
def lstm_model(input_shape):
    batch_size = 8 
    lstm_model = Sequential()
    lstm_model.add(LSTM(7))  
    lstm_model.add(Dense(1))
    lstm_model.compile(loss = 'mean_squared_error', optimizer = "adam", metrics=['mse'])

# Training

    # TODO: Implement your model here. 
    # (Hint: Make use of `Sequential` function from tensorflow.keras to group a linear stack of layers into the model. In addition, 
    # you can apply droupouts to your input using `Dropout` from tensorflow.keras.layers)
    return lstm_model


In [114]:
!python --version

Python 3.8.5


In [105]:

#batch_size
lstm=lstm_model(features_set.shape)
lstm.fit(features_set, labels, batch_size=8, epochs=10, verbose=1)
lstm.summary()

Epoch 1/10


NotImplementedError: in user code:

    /home/A19893678/local/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /home/A19893678/local/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/A19893678/local/miniconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/A19893678/local/miniconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/A19893678/local/miniconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /home

Compile your model, train it and evaluate it. 

What is your mse - and what is your accuracy (note that you have two different types of labels readily available to you). 



Optional: Try your model with more features, try with different sequence lengths (this is a hyperparameter) - and try with the "real" Dax Performance Index data also provided (which reaches back longer). If you do so, you will have to split the data into training and test sets first). 